# Introduction to Tensors and Torch

    In mathematics, a tensor is an arbitrarily complex geometric object that maps in a multi-linear manner geometric vectors, scalars, and other tensors to a resulting tensor.
    
    -- Wikipedia



Here's another excerpt from the Wikipedia article on tensors:

    Although seemingly different, the various approaches to defining tensors describe the same geometric concept using different language and at different levels of abstraction. A tensor may be represented as a (potentially multidimensional) array (although a multidimensional array is not necessarily a representation of a tensor, as discussed below with regard to holors).

In this class, when we say **tensor**, we will just mean **a multidimensional array**. But the term **dimension** can get confusingly overloaded here. Let's see why.

A 1-dimensional tensor is just a vector. In this class, we'll predominantly be using the ```torch``` package for manipulating tensors. In ```torch```, we create a vector as follows:

In [ ]:
from torch import tensor
v = tensor([4, 5, 6, 7, 8])
v

Ok, but then this ends up being described as a vector of dimension 5, or **a 1-dimensional tensor of dimension 5**. Which is super confusing. So typically, we will refer to these separate concepts as the **order** and the **shape**. So this is an **order-1 tensor of shape (5)**. We can get these numbers from the tensor.

In [ ]:
print('The order of the tensor is {}.'.format(v.dim()))
print('The shape of the tensor is {}.'.format(v.shape))

Note that ```.shape``` is an attribute of the tensor, not a method call. You can get parts of vectors in a similar way you would for Python lists.

In [ ]:
print(v[:3])
print(v[2:])
print(v[2:4])

**Exercise:** what's a simple way to extract the vector ```[4, 6, 8]```?

In [ ]:
pass # returns tensor([4, 6, 8])

An important special case is converting an order-1 tensor into a scalar. In ```torch```, they recommend doing this using ```.item()```.

In [ ]:
w = tensor([12])
w.item()

Note that ```.item()``` will throw an exception if you apply it to a non-singleton.

In [ ]:
v.item()

An order-2 tensor is a **matrix**. Here's how we would create a matrix with 4 rows and 3 columns using ```torch```.

In [ ]:
import torch
M = tensor([[4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]])
print(M)
print('The order of M is {}.'.format(M.dim()))
print('The shape of M is {}.'.format(M.shape))

How do we get submatrices? It's sort of like the syntax for getting parts of Python lists, though a bit weird to get used to. Commas are involved.

In [ ]:
print(M) 
print(M[:2]) # first two rows

In [ ]:
print(M) 
print(M[2:]) # last two rows

In [ ]:
print(M) 
print(M[0::2]) # every second row

In [ ]:
print(M) 
print(M[:,:2]) # first two columns

In [ ]:
print(M) 
print(M[:,2:3]) # last column

In [ ]:
print(M) 
print(M[:,0::2]) # every second column

**Exercise:** How do you extract ```tensor([[7, 9], [10, 12]])``` from ```M```?

In [ ]:
print(M)
raise Exception('Complete exercise before proceeding!')

You can do the usual elementwise operations on matrices, and transpose them.

In [ ]:
print(M)
print(M+M)
print(3*M)
print(M*M) # note: this is elementwise multiplication (sometimes called Hadamard product), not matrix multiplication!
print(M.t()) # matrix transpose

You can do standard matrix multiplication with ```torch.mm```.

In [ ]:
print(torch.mm(M, M.t()))

You cannot however multiply a matrix and a vector using `mm`. However there is another function called `mv`.

In [ ]:
print(M)
v = tensor([1, 2, 3])
torch.mm(M, v)

In [ ]:
torch.mv(M, v)

One option is to ```unsqueeze``` the length 3 vector into a 3x1 matrix and then call `mm`.

In [ ]:
v = tensor([1, 2, 3])
print("Before unsqueezing:")
print(v)
v = v.unsqueeze(1)
print("\nAfter unsqueezing:")
print(v)
torch.mm(M, v)

Unsqueezing increases the order of a tensor by 1. The argument to ```unsqueeze``` tells us where to put the new dimension.

In [ ]:
print(f"Original shape of M:  {M.shape}")
M_prime = M.unsqueeze(0)
print(f"After M.unsqueeze(0): {M_prime.shape}")
print(M_prime)
M_prime = M.unsqueeze(1)
print(f"After M.unsqueeze(1): {M_prime.shape}")
print(M_prime)
M_prime = M.unsqueeze(2)
print(f"After M.unsqueeze(2): {M_prime.shape}")
print(M_prime)

If the tensor's shape contains a dimension of size 1, then you can squeeze that dimension to remove it.

In [ ]:
M_prime = tensor([[[ 4,  5,  6],
                   [ 7,  8,  9],
                   [10, 11, 12],
                   [13, 14, 15]]])
print(f"Shape of M_prime: {M_prime.shape}")
M_prime = M_prime.squeeze(0)
print(f"After squeezing:  {M_prime.shape}")

One issue that is likely going to bite you again and again is the datatype of tensors, which tells you what the entries of the tensors are: usually either floating point (of some kind) or integer (of some kind). ```torch``` does its best to guess what you want, but it's not psychic. For the matrix ```M2```, it assumes that because all the initialized entries are integers, then we want a matrix with an integer datatype.

In [ ]:
M2 = tensor([[4,5,6],[2,8,9],[1,7,3]])
print(M2.dtype)

But maybe we wanted these to be floating point, and unsuspectingly try to take the matrix inverse.

In [ ]:
torch.inverse(M2)

Essentially it's complaining that a matrix of datatype ```torch.int64``` (a so-called ```LongTensor```) can't be inverted, because there's no integer matrix ```M^-1``` by which we can multiply it such that ```M * M^-1 = I```. So we need to tell it explicitly that we want to treat matrix ```M``` as a floating-point matrix.

In [ ]:
M2 = M2.float()
M2inv = torch.inverse(M2)
print(M2inv)
I = M2.mm(M2inv)
print(I)

**Exercise:** How do we convert this back to a LongTensor?

In [ ]:
raise Exception('Complete exercise before proceeding!')

```reshape``` is a kind of magical operation that repacks the elements of a matrix into a different shape by copying the elements row by row.

In [ ]:
print(M)
print(M.reshape((2, 6)))
print(M.reshape(12))

```torch``` supports the notion of **broadcasting**, which allows us to conveniently use elementwise operators over tensors of different orders. For instance, if we add a vector ```v``` of shape (3) to a matrix ```M``` of shape (4,3), then ```torch``` assumes that we want to add two (4,3) matrices. One is just ```M```. The other is 4 copies of vector ```v``` piled on top of each other to make another (4,3) matrix. Surprisingly this turns out to be useful. In general, you can use broadcasting to apply elementwise operations if one tensor has shape (a, b, ... n, n + 1, ... m) and the other has shape (n, n + 1, ... m). 

In [ ]:
A = tensor([[4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]])
B = tensor([[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]])
print("A is:")
print(A)
print("B is:")
print(B)
print("A + B is:")
print(A+B)

In [ ]:
A = tensor([[4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]])
b = tensor([1, 2, 3])
print("A is:")
print(A)
print("b is:")
print(b)
print("A + b is:")
print(A+b)

Now let's turn out attention to order-3 tensors. One useful order-3 tensor is an RGB image file. Consider this image of the Swiss flag:

![swiss flag](img/swiss.gif "Swiss flag")

We can view the pixels as a 5x5 matrix. If this were a black-and-white image, it would look like:

In [ ]:
SWISS_FLAG_BW = tensor([[0,0,0,0,0],[0,0,1,0,0],[0,1,1,1,0],[0,0,1,0,0],[0,0,0,0,0]])
print(SWISS_FLAG_BW)

But it isn't a black-and-white image, it's a color image, which means that for each pixel, there's a red value (from 0-255), a green value (from 0-255), and a blue value (from 0-255).

**Exercise:** Represent the Swiss flag as an order-three tensor.

In [ ]:
SWISS_FLAG = pass # TODO: complete this line
print(SWISS_FLAG)
print('The Swiss flag has shape: {}.'.format(SWISS_FLAG.shape))

Sometimes it can be easier to use torch's tensor concatenation operations to construct these.

In [ ]:
SWISS_FLAG_RED = tensor( [ [255,255,255,255,255],
                           [255,255,255,255,255],
                           [255,255,255,255,255],
                           [255,255,255,255,255],
                           [255,255,255,255,255]])

SWISS_FLAG_GREEN = tensor([[0,0,0,0,0],
                           [0,0,255,0,0],
                           [0,255,255,255,0],
                           [0,0,255,0,0],
                           [0,0,0,0,0]])

SWISS_FLAG_BLUE  = tensor([[0,0,0,0,0],
                           [0,0,255,0,0],
                           [0,255,255,255,0],
                           [0,0,255,0,0],
                           [0,0,0,0,0]])

SWISS_FLAG = torch.stack([SWISS_FLAG_RED, SWISS_FLAG_GREEN, SWISS_FLAG_BLUE])
print(SWISS_FLAG)
print('The Swiss flag has shape: {}.'.format(SWISS_FLAG.shape))

We can actually display this image in this notebook using a plotting library called ```matplotlib``` (which, if you don't have, make sure you ```pip install matplotlib```). The problem is, it expects the image tensor to have shape (5,5,3), rather than (3,5,5). In other words, rather than (x,y,color) coordinates, it expects (color,x,y) coordinates. Luckily, higher-order tensors have generalized transpose operations that swap arbitrary axes.

In [ ]:
T = tensor([[[1,2], [3,4]], [[5,6], [7,8]]])
print(f"T has size {T.shape}.")
print("\nThis is T:")
print(T)
print("\nThis is T, after transposing dimensions 0 and 1:")
print(T.transpose(0,1))

In [ ]:
print("This is T:")
print(T)
print("\nThis is T, after transposing dimensions 0 and 2:")
print(T.transpose(0,2))

In [ ]:
print("This is T:")
print(T)
print("\nThis is T, after transposing dimensions 1 and 2:")
print(T.transpose(1,2))

**Exercise:** Convert the Swiss flag into an image tensor.

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline
new_flag = SWISS_FLAG.transpose(0, 1).transpose(1, 2) # TODO: complete this line!
print('The original flag shape: {}.'.format(SWISS_FLAG.shape))
print('The modified flag shape: {}.'.format(new_flag.shape))
imshow(new_flag)


Can you think of an example of an order-3 tensor, famous in pop culture? Hint, its shape is (3, 3, 3).

In [ ]:
example = "RUBIK'S CUBE"

def submit(response):
    import rpyc
    c = rpyc.connect("137.165.10.56", 18861)
    print(c.root.submit_response('lec1', response))

print('You submit the password {} to the server.'.format(example))
submit(example)

**This tutorial is now complete. Please proceed to do Lab 0 ("Rubik") from the Github Classroom:**

https://classroom.github.com/a/X0Cljc2-